## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

In [34]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 11.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from numba import jit, njit
%load_ext line_profiler

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
import random

N = 1000000
A = [random.randint(0, 1000) for _ in range(N)]

In [ ]:
def f1(A):
  acc, cnt = 0,0
  for x in A:
    acc += (x + 100)
    cnt += 1
  return acc / cnt

def f2(A):
  acc = 0
  for x in A:
    acc += (x + 100)
  return acc / len(A)

def f3(A):
  acc = 0
  for x in A:
    acc += x
  return acc / len(A) + 100

@njit
def f4(A):
  acc, cnt = 0,0
  for x in A:
    acc += (x + 100)
    cnt += 1
  return acc / cnt

In [ ]:
%timeit f1(A)

126 ms ± 37.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit f2(A)

92.4 ms ± 32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit f3(A)

44.3 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit f4(A)

1.8 s ± 507 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import string
N = 2000000
df = pd.DataFrame(np.random.randn(N,4), columns = [f'col{i}' for i in range(4)])
df['key'] = np.random.choice(list(string.ascii_letters.lower()), N, replace = True)

In [ ]:
def g1(df):
  res = pd.DataFrame()
  for letter in ['a','b','c','d','e']:
    res = pd.concat([res, df[df['key'] == letter]], axis = 0)
  return res

def g2(df):
  res = pd.concat([df[df['key'] == letter] for letter in ['a','b','c','d','e']], axis = 0)
  return res

@njit
def g3(df):
  res = pd.DataFrame()
  for letter in ['a','b','c','d','e']:
    res = pd.concat([res, df[df['key'] == letter]], axis = 0)
  return res

In [ ]:
%timeit g1(df)

1.09 s ± 310 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit g2(df)

1.24 s ± 584 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%timeit g3(df)

## Лабораторная работа 3

In [ ]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 8.9 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [6]:
import pandas as pd
import time

reviews = pd.read_csv('/content/reviews_sample.csv')
recipes = pd.read_csv('/content/recipes_sample.csv')

<ipython-input-6-d909a92ebec1>:4: DtypeWarning: Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews = pd.read_csv('/content/reviews_sample.csv')


In [7]:
recipes['n_steps'] = pd.to_numeric(recipes['n_steps'], errors='coerce')
recipes['n_ingredients'] = pd.to_numeric(recipes['n_ingredients'], errors='coerce')

reviews['date'] = pd.to_datetime(reviews['date'])
reviews['rating'] = pd.to_numeric(reviews['rating'], errors='coerce')

In [9]:
def mean_rating_a(reviews):
    sum_rating = 0
    count = 0
    for _, row in reviews.iterrows():
        if row['date'].year == 2010:
            sum_rating += row['rating']
            count += 1
    return sum_rating / count if count != 0 else None

start_time = time.time()
mean_a = mean_rating_a(reviews)
print('Средний рейтинг (метод A):', mean_a)
print('Время выполнения (метод A):', time.time() - start_time)

Средний рейтинг (метод A): 4.4544402182900615
Время выполнения (метод A): 6.700645446777344


In [16]:
reviews_2010 = reviews[reviews['date'].dt.year == 2010]

def mean_rating_b(reviews):
    sum_rating = 0
    count = 0
    for _, row in reviews.iterrows():
        sum_rating += row['rating']
        count += 1
    return sum_rating / count if count != 0 else None

mean_b = mean_rating_b(reviews_2010)
print('Средний рейтинг (метод B):', mean_b)
print('Время выполнения (метод B):', time.time() - start_time)

Средний рейтинг (метод B): 4.4544402182900615
Время выполнения (метод B): 45.403834104537964


In [15]:
mean_c = reviews_2010['rating'].mean()
print('Средний рейтинг (метод C):', mean_c)
print('Время выполнения (метод C):', time.time() - start_time)

Средний рейтинг (метод C): 4.4544402182900615
Время выполнения (метод C): 28.819056034088135


In [21]:
#проверка
if mean_a == mean_b == mean_c:
  print('Good job')
else:
  print('Error')

Good job


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

Медленнее всего выполняется функция методом В.

In [32]:
import pandas as pd

df = pd.DataFrame(recipes)

def test_loop():
    for idx, row in df.iterrows():
        print(f"Index: {idx}\nRow:\n{row}\n")

test_loop()

Выходные данные были обрезаны до нескольких последних строк (5000).
minutes                                                          50
contributor_id                                               785369
submitted                                                2008-03-20
n_steps                                                         NaN
description       this delicous weight watcher core program reci...
n_ingredients                                                  11.0
Name: 29583, dtype: object

Index: 29584
Row:
name                          ww core   sweet pork crock pot recipe
id                                                           194331
minutes                                                         250
contributor_id                                               378067
submitted                                                2006-11-06
n_steps                                                         NaN
description       i threw this together the other day and my fam...
n_

 По результатам, основное время работы занято методом iterrows(), что объясняет, почему функция выполняется так медленно.

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [36]:
from line_profiler import LineProfiler 
import time 
import pandas as pd 

In [38]:
reviews = pd.read_csv('reviews_sample.csv', index_col=0, nrows=1000) 
reviews['review'] = reviews['review'].astype("string") 
reviews['date'] = reviews['date'].astype("datetime64[ns]") 
 
def get_word_reviews_count(df): 
    start = time.time() 
    word_reviews = {} 
    for _, row in df.dropna(subset=['review']).iterrows(): 
        recipe_id, review = row['recipe_id'], row['review'] 
        words = review.split(' ') 
        for word in words: 
            if word not in word_reviews: 
                word_reviews[word] = [] 
            word_reviews[word].append(recipe_id) 
     
    word_reviews_count = {} 
    for _, row in df.dropna(subset=['review']).iterrows(): 
        review = row['review'] 
        words = review.split(' ') 
        for word in words: 
            word_reviews_count[word] = len(word_reviews[word]) 
    end = time.time() 
    print('Время выполнения в секундах: ', end - start) 
 
def get_word_reviews_count2(df): 
    start = time.time() 
    word_reviews_count = {} 
    df = df.dropna(subset=['review']) 
    for index in  range(df.shape[0]): 
        review = df['review'].iloc[index] 
        words = review.split(' ') 
        for word in words: 
            if word not in word_reviews_count.keys(): 
                word_reviews_count[word] = 1 
            else: word_reviews_count[word] = word_reviews_count[word] + 1 
    end = time.time() 
    print('Время выполнения в секундах: ', end - start)  
 
get_word_reviews_count(reviews) 
get_word_reviews_count2(reviews)

Время выполнения в секундах:  0.14696145057678223
Время выполнения в секундах:  0.03000044822692871


4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [41]:
import numpy as np
import numba as nb

In [42]:
def mape(reviews):
    total_rating = 0
    count = 0
    for rating in reviews:
        if rating != 0:
            total_rating += abs(rating - np.mean(reviews))
            count += 1
    return (total_rating / count) / np.mean(reviews) * 100

In [51]:
from numba import jit

@jit(nopython=True)
def mape_numba(reviews):
    total_rating = 0
    count = 0
    for rating in reviews:
        if rating != 0:
            total_rating += abs(rating - np.mean(reviews))
            count += 1
    return (total_rating / count) / np.mean(reviews) * 100

In [23]:
def mape_numpy(reviews):
    reviews = reviews[reviews != 0]
    return (np.mean(np.abs(reviews - np.mean(reviews))) / np.mean(reviews)) * 100

In [48]:
from numba import jit
@jit(nopython=True)
def mape_numpy_numba(reviews):
    reviews = reviews[reviews != 0]
    return (np.mean(np.abs(reviews - np.mean(reviews))) / np.mean(reviews)) * 100

In [52]:
import timeit

reviews = np.random.randint(0, 5, size=1000)

print("Без использования векторизованных операций и методов массивов numpy и без использования numba:", timeit.timeit(lambda: mape(reviews), number=100))
print("Без использования векторизованных операций и методов массивов numpy, но с использованием numba:", timeit.timeit(lambda: mape_numba(reviews), number=100))
print("С использованием векторизованных операций и методов массивов numpy, но без использования numba:", timeit.timeit(lambda: mape_numpy(reviews), number=100))
print("C использованием векторизованных операций и методов массивов numpy и numba:", timeit.timeit(lambda: mape_numpy_numba(reviews), number=100))

Без использования векторизованных операций и методов массивов numpy и без использования numba: 1.4753439490000346
Без использования векторизованных операций и методов массивов numpy, но с использованием numba: 0.2647920869999325
С использованием векторизованных операций и методов массивов numpy, но без использования numba: 0.006925797999883798
C использованием векторизованных операций и методов массивов numpy и numba: 0.0006624130001000594
